In [12]:
import sys
sys.executable

'/home/lisheard/miniconda3/envs/cosmospy37/bin/python'

In [22]:
from dotenv import dotenv_values

config = dotenv_values('./.env')
#config = dotenv_values('./.env_serverless')

In [23]:
serverless = config['MODE']=='serverless'
serverless

False

In [25]:
from azure.cosmos import CosmosClient

import os
#url = os.environ['ACCOUNT_URI']
#key = os.environ['ACCOUNT_KEY']
url = config['ACCOUNT_URI']
key = config['ACCOUNT_KEY']
client = CosmosClient(url, credential=key)
url

'https://explorecosmos.documents.azure.com:443/'

In [28]:
database = client.get_database_client('docs_db')
container = database.get_container_client('PSC-sample')
#container = database.get_container_client('PSC')    # populated with 500k records from ADF

In [5]:
from azure.cosmos import exceptions
import os

database_name = 'docs_db'
try:
    database = client.create_database(database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database_name)

In [6]:
from azure.cosmos import PartitionKey
container_name = 'PSC-sample'

try:
    container = database.create_container(id=container_name, partition_key=PartitionKey(path="/company_number"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)
except exceptions.CosmosHttpResponseError:
    raise

In [7]:
# Insert jsonlines data

In [8]:
container.read()

{'id': 'PSC-sample',
 'indexingPolicy': {'indexingMode': 'consistent',
  'automatic': True,
  'includedPaths': [{'path': '/*'}],
  'excludedPaths': [{'path': '/"_etag"/?'}]},
 'partitionKey': {'paths': ['/company_number'], 'kind': 'Hash', 'version': 2},
 'conflictResolutionPolicy': {'mode': 'LastWriterWins',
  'conflictResolutionPath': '/_ts',
  'conflictResolutionProcedure': ''},
 'geospatialConfig': {'type': 'Geography'},
 '_rid': 'MVhlAOI57UE=',
 '_ts': 1633634034,
 '_self': 'dbs/MVhlAA==/colls/MVhlAOI57UE=/',
 '_etag': '"0000209a-0000-1100-0000-615f46f20000"',
 '_docs': 'docs/',
 '_sprocs': 'sprocs/',
 '_triggers': 'triggers/',
 '_udfs': 'udfs/',
 '_conflicts': 'conflicts/'}

In [9]:
indexing_off = {
    'indexingMode': 'consistent',
    'automatic': True,
    'excludedPaths': [{'path': '/*'}]}

In [10]:
indexing_on = {
    'indexingMode': 'consistent',
    'automatic': True,
    'includedPaths': [{'path': '/*'}],
    'excludedPaths': [{'path': '/"_etag"/?'}]}

In [11]:
response = database.replace_container(container, container.read()['partitionKey'], indexing_off)

In [12]:
container.read(populate_quota_info = True,
                      response_hook = lambda h,p: print(h['x-ms-documentdb-collection-index-transformation-progress']))

100


{'id': 'PSC-sample',
 'indexingPolicy': {'indexingMode': 'consistent',
  'automatic': True,
  'includedPaths': [],
  'excludedPaths': [{'path': '/*'}, {'path': '/"_etag"/?'}]},
 'partitionKey': {'paths': ['/company_number'], 'kind': 'Hash', 'version': 2},
 'conflictResolutionPolicy': {'mode': 'LastWriterWins',
  'conflictResolutionPath': '/_ts',
  'conflictResolutionProcedure': ''},
 'geospatialConfig': {'type': 'Geography'},
 '_rid': 'MVhlAOI57UE=',
 '_ts': 1633634053,
 '_self': 'dbs/MVhlAA==/colls/MVhlAOI57UE=/',
 '_etag': '"0000249a-0000-1100-0000-615f47050000"',
 '_docs': 'docs/',
 '_sprocs': 'sprocs/',
 '_triggers': 'triggers/',
 '_udfs': 'udfs/',
 '_conflicts': 'conflicts/'}

In [19]:
if not serverless:
    container.replace_throughput(4000).properties

In [14]:
from ratelimit import limits, sleep_and_retry

# throttle the upsert request to 100 per second
# we now have 4000 RU per second, and one insertion is ~12 RU
# so throttle to 300 calls per second
# this should mean inserting 500k docs in <30 minutes without 429 errors

@sleep_and_retry
@limits(calls=300, period=1)
def throttled_upsert(doc):
    container.upsert_item(doc)
    #print(container.client_connection.last_response_headers['x-ms-request-charge'])


In [70]:
#throttled_upsert(sampledoc)

10.29


In [ ]:
import jsonlines

#data = './sampledata/psc-sample.jsonl'
data = './sampledata/psc-snapshot-2021-10-07_1of20.txt'
i = 0
with jsonlines.open(data) as reader:
    for obj in reader:
        i+=1
        obj['id']=str(hash(obj['data']['links']['self']))
        if serverless:
            container.upsert_item(obj)
        else: 
            throttled_upsert(obj)
        if i % 1000 == 0:
            print(f'{i} objects processed')

In [20]:
# scale back down after import
if not serverless:
    container.replace_throughput(400).properties

# Query Database

In [33]:
# Count records
q = 'SELECT VALUE COUNT(1) FROM PSC'
res = container.query_items(
        query=q,
        enable_cross_partition_query=True)
print(list(res)[0], 'records')


430934 records


In [68]:
q = 'SELECT * FROM PSC'
q = 'SELECT TOP 1 * FROM PSC'
res = container.query_items(
        query=q,
        enable_cross_partition_query=True)
#print(len(list(res)), 'records')
sampledoc = list(res)[0]
sampledoc

{'company_number': '07328553',
 'data': {'address': {'address_line_1': 'Goosander Way',
   'locality': 'London',
   'postal_code': 'SE28 0ER',
   'premises': '5'},
  'ceased_on': '2017-09-01',
  'country_of_residence': 'England',
  'date_of_birth': {'month': 6, 'year': 1981},
  'etag': 'bc02fa3138d0c2de20ac569bcffe41e320bcf3a1',
  'kind': 'individual-person-with-significant-control',
  'links': {'self': '/company/07328553/persons-with-significant-control/individual/H5NJcJ-5crTRDoYTBL6Y9GNvGIU'},
  'name': 'Mrs Charlotte Eseku',
  'name_elements': {'forename': 'Charlotte',
   'surname': 'Eseku',
   'title': 'Mrs'},
  'nationality': 'British',
  'natures_of_control': ['ownership-of-shares-25-to-50-percent'],
  'notified_on': '2016-06-30'},
 'id': '432122976140361427',
 '_rid': 'V9tpAPHLFywrAAAAAAAAAA==',
 '_self': 'dbs/V9tpAA==/colls/V9tpAPHLFyw=/docs/V9tpAPHLFywrAAAAAAAAAA==/',
 '_etag': '"0300ecd2-0000-1100-0000-615f2d3c0000"',
 '_attachments': 'attachments/',
 '_ts': 1633627452}

In [10]:
client.client_connection.last_response_headers['x-ms-request-charge']

'2.89'

# Cleanup

In [90]:
# delete container
database.delete_container(container)